In [1]:
# Imports
from PIL import Image
from ultralytics import YOLO
import json
import numpy as np
import matplotlib.path as mpltPath
import csv
import matplotlib.pyplot as plt
import sys
import matplotlib.colors as mcolors
import cv2
import statistics

model_path = r"S:\Phys\FIV942 Adipo\Datasets\02h9\YO 512 0320 Snow\map75=0607616 YOLOv8n-segml idx=5 ep=52 btch=32 rnd=4670342\weights\best.pt"
val_data = r"S:\Phys\FIV942 Adipo\FIV942P3 Chops 512"

adipo_seg = YOLO(model_path)

predictions = adipo_seg.predict(val_data, show=False, max_det = 1000)

img1_data = predictions[0]
img1_mask_coords = img1_data.masks.xy
img1_classes = img1_data.boxes.cls.numpy()

class0_indices = np.where(img1_classes == 0)[0]

class1_indices = np.where(img1_classes == 1)[0]

img1_box_coords = img1_data.boxes.xyxy
img1_class0_bbox_coords = img1_box_coords[class0_indices]
img1_class1_bbox_coords = img1_box_coords[class1_indices]

def polygon_area(coords):
    """
    Calculate the area of a polygon given its vertices using numpy.
    :param coords: A numpy array of shape (n, 2), where n is the number of vertices.
    :return: The area of the polygon.
    """
    x = coords[:, 0]
    y = coords[:, 1]
    i=np.arange(len(x))
    # 'shoelace' formula
    # return 0.5*np.abs(np.dot(x, np.roll(y, -1)) - np.dot(y, np.roll(x, -1)))
    return np.abs(np.sum(x[i-1]*y[i]-x[i]*y[i-1])*0.5)

# Example list of numpy arrays as provided
polygons = img1_mask_coords

# Calculate the area for each polygon and store the results in a new list
areas = [polygon_area(polygon) for polygon in polygons]

class1_parents = []

for class1_coord in img1_class1_bbox_coords.numpy():
    class1_instance_parents = []
    class0_id = 0
    print("the class1 instance: ", class1_coord)
    for class0_coord in img1_class0_bbox_coords.numpy():
        print("the class0 instance: ", class0_coord)
        if class1_coord[0] > class0_coord[2] or class1_coord[2] < class0_coord[0] or class1_coord[1] > class0_coord[3] or class1_coord[3] < class0_coord[1]:
            print("it wasn't a parent")
            class0_id += 1
            print("class0_id is now: ", class0_id)
            continue
        elif (class1_coord[0] > class0_coord[0] and class1_coord[0] < class0_coord[2] and class1_coord[1] > class0_coord[1] and class1_coord[1] < class0_coord[3]) or (class1_coord[2] < class0_coord[2] and class1_coord[2] > class0_coord[0] and class1_coord[3] > class0_coord[1] and class1_coord[3] < class0_coord[3]):
            class1_instance_parents.append(class0_indices[class0_id])
            print("it was an instance, we added ", class0_indices[class0_id])
            class0_id += 1
            print("now class0_id is: ", class0_id)
            
    class1_parents.append(class1_instance_parents)
    
image_path = r"S:\Phys\FIV942 Adipo\Adipo2Viz 512\001597.bmp"
image = Image.open(image_path)
image_array = np.array(image)

pixel_intensity_data = []
for mask_outline in range(len(img1_mask_coords)):
    vertices = img1_mask_coords[mask_outline]

    
    # Assuming image_array is a 2D numpy array for a grayscale image
    # For a color image, you need to decide how to handle the color channels

    # Create a mesh grid of coordinate values
    xx, yy = np.meshgrid(np.arange(image_array.shape[1]), np.arange(image_array.shape[0]))

    # Flatten the grid arrays
    x_flat = xx.flatten()
    y_flat = yy.flatten()

    # Create a list of (x, y) points from the flattened grid
    class1_points = np.vstack((x_flat, y_flat)).T

    # Create a path object from the vertices
    polygon_path = mpltPath.Path(vertices)

    # Use the path object to create a mask
    inside_polygon = polygon_path.contains_points(class1_points)

    # Reshape the mask back to the image shape
    mask = inside_polygon.reshape(xx.shape)

    # Apply the mask to select pixels within the polygon
    # selected_pixels = image_array[mask]

    # Sum the intensities of the selected pixels
    # sum_of_intensities = np.sum(selected_pixels)

    # print("Sum of pixel intensities in the polygonal region:", sum_of_intensities)
    # Initialize an array to hold the sum of intensities for each channel
    sum_of_intensities_per_channel = np.zeros(image_array.shape[2])

    # Iterate over each channel
    for i in range(image_array.shape[2]):
        # Apply the mask to the current channel and sum the intensities
        selected_pixels = image_array[:, :, i][mask]
        sum_of_intensities_per_channel[i] = np.sum(selected_pixels)

    print("Sum of pixel intensities in the polygonal region for each channel:", sum_of_intensities_per_channel)
    pixel_intensity_data.append(sum_of_intensities_per_channel)
    
img1_box_centers = img1_data.boxes.xywh.numpy()

box_intensity_data = []
for box_instance in range(len(img1_box_centers)):
    bbox_xywh = img1_data.boxes.xywh.numpy()[box_instance]
    bbox_corners = [[bbox_xywh[0] - bbox_xywh[2], bbox_xywh[1] + bbox_xywh[3]],[bbox_xywh[0] + bbox_xywh[2], bbox_xywh[1] + bbox_xywh[3]]
                , [bbox_xywh[0] + bbox_xywh[2], bbox_xywh[1] - bbox_xywh[3]], [bbox_xywh[0] - bbox_xywh[2], bbox_xywh[1] - bbox_xywh[3]]]
    vertices = bbox_corners

    
    # Assuming image_array is a 2D numpy array for a grayscale image
    # For a color image, you need to decide how to handle the color channels

    # Create a mesh grid of coordinate values
    xx, yy = np.meshgrid(np.arange(image_array.shape[1]), np.arange(image_array.shape[0]))

    # Flatten the grid arrays
    x_flat = xx.flatten()
    y_flat = yy.flatten()

    # Create a list of (x, y) points from the flattened grid
    class1_points = np.vstack((x_flat, y_flat)).T

    # Create a path object from the vertices
    polygon_path = mpltPath.Path(vertices)

    # Use the path object to create a mask
    inside_polygon = polygon_path.contains_points(class1_points)

    # Reshape the mask back to the image shape
    mask = inside_polygon.reshape(xx.shape)

    # Apply the mask to select pixels within the polygon
    # selected_pixels = image_array[mask]

    # Sum the intensities of the selected pixels
    # sum_of_intensities = np.sum(selected_pixels)

    # print("Sum of pixel intensities in the polygonal region:", sum_of_intensities)
    # Initialize an array to hold the sum of intensities for each channel
    sum_of_intensities_per_channel = np.zeros(image_array.shape[2])

    # Iterate over each channel
    for i in range(image_array.shape[2]):
        # Apply the mask to the current channel and sum the intensities
        selected_pixels = image_array[:, :, i][mask]
        sum_of_intensities_per_channel[i] = np.sum(selected_pixels)

    print("Sum of pixel intensities in the polygonal region for each channel:", sum_of_intensities_per_channel)
    box_intensity_data.append(sum_of_intensities_per_channel)
    
    # need to add confidence score
header = ["FilePath", "PolyID","cls","Vertices","Number of Vertices","PolyArea","Sum Inten WV0","SumInten WV1", "SumInten WV2","BBox x","BBox y","BBox w","BBox h",
          "BBox intensity WV0", "BBox intensity WV1","BBox intensity WV2","Conf","PolySum", "PolyAvg", "PolyStDDev", "BoxSum", "BoxAvg", "BoxStDDev", "Parents"]
data = []

class1_counter = 0
for i in range(len(img1_classes)):
    # excluding nonexistent masks
    # if img1_mask_coords[i].size == 0:
    #     if img1_classes[i] == 1.0:
    #         class1_counter += 1
    #         print("the class of this mask is 1.0, class1 counter is now ", class1_counter)
    #     continue
    # else:
    cls = predictions[0].boxes[i].cls.numpy().item(0)
    vertices = polygons[i]
    numVertices = len(vertices)
    polyArea = areas[i]
    bbox_data = predictions[0].boxes[i].xywh.numpy()
    conf = predictions[0].boxes.conf.numpy().item(i)
    polySum = sum([pixel_intensity_data[i][0],pixel_intensity_data[i][1],pixel_intensity_data[i][2]])
    polyAvg = polySum / 3
    polyStdDev = statistics.pstdev([pixel_intensity_data[i][0],pixel_intensity_data[i][1],pixel_intensity_data[i][2]])
    boxSum = sum([box_intensity_data[i][0],box_intensity_data[i][1],box_intensity_data[i][2]])
    boxAvg = boxSum / 3
    boxStdDev = statistics.pstdev([box_intensity_data[i][0],box_intensity_data[i][1],box_intensity_data[i][2]])
    polygon_data = [image_path,i,cls,vertices,numVertices,polyArea,pixel_intensity_data[i][0],pixel_intensity_data[i][1],pixel_intensity_data[i][2]
                    , bbox_data.item(0), bbox_data.item(1), bbox_data.item(2), bbox_data.item(3),box_intensity_data[i][0],box_intensity_data[i][1],box_intensity_data[i][2]
                    ,conf, polySum, polyAvg, polyStdDev, boxSum, boxAvg, boxStdDev]
    if class1_indices.__contains__(i):
        # print(i)
        polygon_data.append(class1_parents[class1_counter])
        print("class1 index before increment: ", class1_counter)
        class1_counter += 1
        print("class1 index after increment: ", class1_counter)
    data.append(polygon_data)
    

filename = 'polygon_image_data_test.csv'
with open(filename, 'w', newline="") as file:
    csvwriter = csv.writer(file)
    csvwriter.writerow(header)
    csvwriter.writerows(data)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/3325 S:\Phys\FIV942 Adipo\FIV942P3 Chops 512\FIV942P3-5_FIV942P3.A - 10_0_0_0.bmp: 512x512 (no detections), 79.5ms
image 2/3325 S:\Phys\FIV942 Adipo\FIV942P3 Chops 512\FIV942P3-5_FIV942P3.A - 10_10_0_0.bmp: 512x512 7 Cells, 58 Droplets, 56.6ms
image 3/3325 S:\Phys\FIV942 Adipo\FIV942P3 Chops 512\FIV942P3-5_FIV942P3.A - 10_11_0_0.bmp: 512x512 2 Cells, 9 Droplets, 61.1ms
image 4/3325 S:\Phys\FIV942 Adipo\FIV942P3 Chops 512\FIV942P3-5_FIV942P3.A -

KeyboardInterrupt: 